In [28]:
import torch
from transformers import AutoTokenizer, AutoModel
import os
import json

with open('config.json') as f:
    CONFIG = json.load(f)

MODEL_PATH = CONFIG['model_path']
MODEL_DIR_SUP = CONFIG['model_dir_super']
MODEL_DIR_CAT = CONFIG['model_dir_cat']
DEVICE = torch.device(CONFIG.get('device', 'cuda' if torch.cuda.is_available() else 'cpu'))

def load_mappings(path):
    mapping = {}
    with open(path, encoding="utf-8") as f:
        for line in f:
            if ":" in line:
                idx, label = line.strip().split(":", 1)
                mapping[int(idx)] = label.strip()
    return mapping

idx2sup = load_mappings(os.path.join(MODEL_DIR_SUP, "label_mappings.txt"))
idx2cat = load_mappings(os.path.join(MODEL_DIR_CAT, "label_mappings.txt"))

class ClassifierModel(torch.nn.Module):
    def __init__(self, model_path, nlabels):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.encoder = AutoModel.from_pretrained(model_path, add_pooling_layer=False)
        dim = self.encoder.config.hidden_size
        self.head = torch.nn.Sequential(
            torch.nn.Linear(dim, 256), torch.nn.ReLU(), torch.nn.Dropout(0.2), torch.nn.Linear(256, nlabels)
        )

    def forward(self, texts):
        texts = ["descrizione tecnica: " + t for t in texts]
        tokens = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(DEVICE)
        output = self.encoder(**tokens)
        emb = output.last_hidden_state[:, 0]
        return self.head(emb)

# Load models
sup_model = ClassifierModel(MODEL_PATH, len(idx2sup)).to(DEVICE)
sup_model.load_state_dict(torch.load(os.path.join(MODEL_DIR_SUP, "best_model.pt"), map_location=DEVICE))
sup_model.eval()

cat_model = ClassifierModel(MODEL_PATH, len(idx2cat)).to(DEVICE)
cat_model.load_state_dict(torch.load(os.path.join(MODEL_DIR_CAT, "best_model.pt"), map_location=DEVICE))
cat_model.eval()

def predict(text):
    with torch.no_grad():
        out_sup = sup_model([text])
        pred_sup = out_sup.argmax(dim=1).item()

        out_cat = cat_model([text])
        pred_cat = out_cat.argmax(dim=1).item()

    return idx2sup[pred_sup], idx2cat[pred_cat]

if __name__ == "__main__":
    descrizione = """

ISOLAMENTI TERMICI CAM
ISOLAMENTO TERMICO - CAM - su pareti verticali, orizzontali o inclinate
Compreso:
- oneri per la preparazione dei supporti (se non diversamente esplicitato)
- calo ed il sollevamento dei materiali
- ponti di servizio e/o trabattelli a norma con altezza massima 2,00 m, anche esterni, mobili o fissi
- formazione dei giunti
- raccordo agli angoli,
Le opere devono essere conformi ai Decreti del MASE sui Criteri Ambientali Minimi - CAM, alle prescrizioni delle ditte produttrici e alle normative di riferimento.
I prodotti per l'isolamento devono:
- recare la marcatura CE
- essere conformi alla UNI 13172
- rispettare i requisiti della UNI 13501 in materia di reazione al fuoco da D.M. 15/03/2005 e seguenti (quando previsto).

REALIZZAZIONE DI ISOLAMENTO A CAPPOTTO SU SUPERFICI INTERNE "DISPERDENTI" OPACHE VERTICALI RETTE, rispondenti alle normative tecniche vigenti previste dalle linee guida ETAG e dalle norme UNI, rispondente ai requisiti CAM e composto da una serie di elementi costruttivi (inclusi nel prezzo), realizzato su qualunque supporto ad esclusione del legno, quali: il primer, il collante per l’ancoraggio, il materiale isolante, i tasselli (se necessari), se non diversamente precisato, il rasante, l'armatura con rete in tessuto di fibra di vetro, due mani di primer, strato di intonaco di fondo steso a spatola di almeno 3 mm di finitura (rivestimento con eventuale fondo adatto al sistema) e gli accessori, (se non diversamente precisato) in base alla tipologia di materiale. E’ compresa altresì la verifica del supporto, sono esclusi, quindi da computare a parte, l’eventuale realizzazione del sottofondo o il grosso ripristino dell’intonaco della parete e lo strato di collante, tutte le opere di adeguamento degli impianti elettrici (placche, interruttori, prese e scatole di derivazione etc), gli impianti termici e/o meccanici (radiatori, fancoiler, split interni etc), come la rimozione e nuova fornitura del battiscopa e la tinteggiatura delle pareti o i ripristini degli imbotti delle aperture interne.

Fornitura e posa in opera di pannelli di isolante in Poliuretano espanso (PUR) dello spessore mm 60 conforme alla norma UNI EN 13165:2016, esente da CFC o HCF, abbinato a pannelli di cartongesso dello spessore mm 13, dello spessore complessivo di mm 73, in classe E di reazione al fuoco secondo UNI EN 13501-1:2009; completi degli accessori di ancoraggio, dei sfridi e dei tagli a misura e della rifinitura dei pannelli a regola d'arte, la stuccatura dei giunti con armatura con rete in tessuto di fibra di vetro, la rasatura delle superfici e la successiva tinteggiatura estesa alle riprese delle pareti adiacenti non isolate, sono escluse tutte le riquadrature dovute a sporgenze, nicchie, lesene, ed è altresì escluso il rasante, l’ulteriore armatura con rete in tessuto di fibra di vetro, come le due mani di primer e la stesa dello strato di intonaco di fondo. Per altezze della parete fino a 4 m dal piano di calpestio."""
    sup, cat = predict(descrizione)
    print("Supercategoria:", sup)
    print("Categoria:", cat)


Supercategoria: Massetti, sottofondi, drenaggi, vespai
Categoria: Porte tagliafuoco
